<a href="https://colab.research.google.com/github/karthikeyanbu/GenAI/blob/main/Karthikeyan_GenAI_03Feb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## About the program
# Download customer dataset from Kaggle and use the same for data cleansing demo

In [122]:
# Installations
!apt-get install wget
!wget https://raw.githubusercontent.com/karthikeyanbu/GenAI/main/twcs.csv

!pip install emoji

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
--2024-02-04 17:43:00--  https://raw.githubusercontent.com/karthikeyanbu/GenAI/main/twcs.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20787426 (20M) [text/plain]
Saving to: ‘twcs.csv.2’

twcs.csv.2          100%[===================>]  19.82M  --.-KB/s    in 0.1s    

2024-02-04 17:43:01 (148 MB/s) - ‘twcs.csv.2’ saved [20787426/20787426]



In [123]:
# Libraries
import pandas as pd
import string as str
import re
import nltk
from nltk.corpus import stopwords
import emoji

In [124]:
# Read csv file
# Low_memory = False parameter set as per of debug message
customer_df = pd.read_csv("twcs.csv", low_memory=False)

In [125]:
customer_df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1.0,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2.0,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3.0,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4.0,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5.0,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [126]:
customer_df.shape

(152758, 7)

In [127]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152758 entries, 0 to 152757
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tweet_id                 109227 non-null  float64
 1   author_id                109227 non-null  object 
 2   inbound                  109227 non-null  object 
 3   created_at               109227 non-null  object 
 4   text                     109227 non-null  object 
 5   response_tweet_id        73904 non-null   object 
 6   in_response_to_tweet_id  81056 non-null   float64
dtypes: float64(2), object(5)
memory usage: 8.2+ MB


In [129]:
## text is the field taken for data cleansing
# 1. Converting to lowercase
customer_df["text"]
customer_df["text"] = customer_df["text"].str.lower()
customer_df["text"]

0         @115712 i understand. i would like to assist y...
1             @sprintcare and how do you propose we do that
2         @sprintcare i have sent several private messag...
3         @115712 please send us a private message so th...
4                                        @sprintcare i did.
                                ...                        
152753                                                  NaN
152754                                                  NaN
152755                                                  NaN
152756                                                  NaN
152757                                                  NaN
Name: text, Length: 152758, dtype: object

In [139]:
# Check for the presence of punctuation
customer_df["text"].dtypes
customer_df["text"] = customer_df["text"].to_string()


In [ ]:
for i in customer_df["text"]:
  if ("<html>" in i):
     print (i)

In [ ]:
for i in customer_df["text"]:
  if ("." in i):
     print (i)

In [92]:
#2 Remove punctuations
exclude = type(str.punctuation)

In [93]:
def remove_punc(text):
    if not isinstance(text, exclude):
        return ''
    return re.sub(r'[^\w\s]', '', text)

In [94]:
customer_df["text"] = customer_df["text"].apply(remove_punc)
customer_df["text"].head(10)

0    115712 i understand i would like to assist you...
1         sprintcare and how do you propose we do that
2    sprintcare i have sent several private message...
3    115712 please send us a private message so tha...
4                                     sprintcare i did
5    115712 can you please send us a private messag...
6             sprintcare is the worst customer service
7    115713 this is saddening to hear please shoot ...
8    sprintcare you gonna magically change your con...
9    115713 we understand your concerns and wed lik...
Name: text, dtype: object

In [96]:
#3 Remove URL, if any
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub("",text)

In [99]:
customer_df["text"] = customer_df["text"].apply(remove_url)
customer_df["text"].head(10)

0    115712 i understand i would like to assist you...
1         sprintcare and how do you propose we do that
2    sprintcare i have sent several private message...
3    115712 please send us a private message so tha...
4                                     sprintcare i did
5    115712 can you please send us a private messag...
6             sprintcare is the worst customer service
7    115713 this is saddening to hear please shoot ...
8    sprintcare you gonna magically change your con...
9    115713 we understand your concerns and wed lik...
Name: text, dtype: object

In [115]:
#4 Remove stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [110]:
def remove_stopwords(text):
    new_text=[]
    for word in text.split():
        if word in stopwords.words("english"):
            new_text.append("")
        else:
            new_text.append(word.strip())

    return " ".join(new_text).replace("   ","")

In [111]:
customer_df["text"] = customer_df["text"].apply(remove_stopwords)
customer_df["text"].head(10)

0    115712  understand  would like  assistwould ne...
1                                  sprintcare  propose
2    sprintcaresent several private messagesone  re...
3    115712 please send us  private messageassistcl...
4                                         sprintcare  
5    115712please send us  private message  gain  d...
6                     sprintcareworst customer service
7    115713saddening  hear please shoot us  dm  loo...
8    sprintcare  gonna magically change  connectivi...
9    115713  understand  concerns  wed like please ...
Name: text, dtype: object

In [113]:
#5 Remove html tags, if any
def remove_html_tags(text):
  pattern=re.compile('<.*?>')
  return pattern.sub("",text)

In [114]:
customer_df["text"] = customer_df["text"].apply(remove_html_tags)
customer_df["text"].head(10)

0    115712  understand  would like  assistwould ne...
1                                  sprintcare  propose
2    sprintcaresent several private messagesone  re...
3    115712 please send us  private messageassistcl...
4                                         sprintcare  
5    115712please send us  private message  gain  d...
6                     sprintcareworst customer service
7    115713saddening  hear please shoot us  dm  loo...
8    sprintcare  gonna magically change  connectivi...
9    115713  understand  concerns  wed like please ...
Name: text, dtype: object

In [116]:
#6 Remove emoji, if any
def remove_emoji(text):
    clean_text=emoji.demojize(text)
    return clean_text

In [121]:
customer_df["text"] = customer_df["text"].apply(remove_emoji)
customer_df["text"].head(10)

0    115712  understand  would like  assistwould ne...
1                                  sprintcare  propose
2    sprintcaresent several private messagesone  re...
3    115712 please send us  private messageassistcl...
4                                         sprintcare  
5    115712please send us  private message  gain  d...
6                     sprintcareworst customer service
7    115713saddening  hear please shoot us  dm  loo...
8    sprintcare  gonna magically change  connectivi...
9    115713  understand  concerns  wed like please ...
Name: text, dtype: object